# Question 1

In [6]:
import numpy as np
from scipy.optimize import linprog

np.random.seed(42)

class PricingGame:
    def __init__(self, num_firms=3):
        self.num_firms = num_firms
        self.costs = np.random.uniform(10, 30, num_firms)  # Production costs for each firm
        self.max_price = 100  # Maximum possible price
        self.market_size = 1000  # Total market size
        
        # Price sensitivity coefficients (randomly generated)
        self.price_sensitivity = np.random.uniform(0.5, 1.5, (num_firms, num_firms))
        np.fill_diagonal(self.price_sensitivity, np.random.uniform(1.5, 2.5, num_firms))
    
    def demand(self, prices):
        """Calculate demand for each firm given the prices"""
        base_demand = self.market_size / self.num_firms
        demands = np.full(self.num_firms, base_demand)
        for i in range(self.num_firms):
            for j in range(self.num_firms):
                if i != j:
                    demands[i] += self.price_sensitivity[i, j] * (prices[j] - prices[i])
        return np.maximum(demands, 0)  # Demand cannot be negative
    
    def profit(self, prices):
        """Calculate profit for each firm given the prices"""
        demands = self.demand(prices)
        return (prices - self.costs) * demands
    
    def best_response(self, other_prices, firm_index):
        """Find the best response price for a given firm"""
        def objective(x):
            prices = np.copy(other_prices)
            prices[firm_index] = x[0]
            return -self.profit(prices)[firm_index]  # Negative for maximization
        
        res = linprog(
            c=[1],
            A_ub=[[-1], [1]],
            b_ub=[-self.costs[firm_index], self.max_price],
            method='highs'
        )
        return res.x[0]
    
    def find_nash_equilibrium(self, max_iterations=100, tolerance=1e-6):
        """Find the Nash equilibrium prices using iterative best response"""
        prices = np.random.uniform(self.costs, self.max_price, self.num_firms)
        for _ in range(max_iterations):
            old_prices = np.copy(prices)
            for i in range(self.num_firms):
                other_prices = np.copy(prices)
                prices[i] = self.best_response(other_prices, i)
            if np.all(np.abs(prices - old_prices) < tolerance):
                break
        return prices

# Run the simulation
game = PricingGame()
nash_prices = game.find_nash_equilibrium()

print("Firm costs:")
for i, cost in enumerate(game.costs):
    print(f"Firm {i+1}: {cost:.2f}")

print("\nNash equilibrium prices:")
for i, price in enumerate(nash_prices):
    print(f"Firm {i+1}: {price:.2f}")

demands = game.demand(nash_prices)
profits = game.profit(nash_prices)

print("\nDemand at equilibrium:")
for i, demand in enumerate(demands):
    print(f"Firm {i+1}: {demand:.2f}")

print("\nProfit at equilibrium:")
for i, profit in enumerate(profits):
    print(f"Firm {i+1}: {profit:.2f}")

Firm costs:
Firm 1: 17.49
Firm 2: 29.01
Firm 3: 24.64

Nash equilibrium prices:
Firm 1: 17.49
Firm 2: 29.01
Firm 3: 24.64

Demand at equilibrium:
Firm 1: 345.58
Firm 2: 322.09
Firm 3: 326.97

Profit at equilibrium:
Firm 1: 0.00
Firm 2: 0.00
Firm 3: 0.00


# Question 2

In [18]:
import numpy as np
from scipy.optimize import minimize

np.random.seed(42)

class AdGame:
    def __init__(self):
        self.market_size = 1000000
        self.base_shares = np.array([0.4, 0.35, 0.25])
        self.ad_effectiveness = np.array([0.1, 0.12, 0.15])
        self.cost_coefficient = np.array([1.2, 1.0, 0.8])
        self.max_budget = 100000

    def market_share(self, ad_spends):
        weighted_spends = self.ad_effectiveness * ad_spends
        total_effect = np.sum(weighted_spends)
        if total_effect == 0:
            return self.base_shares
        return self.base_shares + (weighted_spends / total_effect) * (1 - np.sum(self.base_shares))

    def profit(self, ad_spends):
        shares = self.market_share(ad_spends)
        revenues = shares * self.market_size
        costs = self.cost_coefficient * ad_spends
        return revenues - costs

    def best_response(self, other_spends, company_index):
        def neg_profit(x):
            spends = np.copy(other_spends)
            spends[company_index] = x[0]
            return -self.profit(spends)[company_index]

        res = minimize(neg_profit, [self.max_budget/2], bounds=[(0, self.max_budget)], method='L-BFGS-B')
        return res.x[0]

    def find_nash_equilibrium(self, max_iterations=1000, tolerance=1e-6):
        spends = np.array([48222.41, 52894.19, 60170.29]) 
        for _ in range(max_iterations):
            old_spends = np.copy(spends)
            for i in range(3):
                other_spends = np.copy(spends)
                spends[i] = self.best_response(other_spends, i)
            if np.all(np.abs(spends - old_spends) < tolerance):
                break
        return spends

game = AdGame()
nash_spends = game.find_nash_equilibrium()
shares = game.market_share(nash_spends)
profits = game.profit(nash_spends)




Nash Equilibrium Advertising Spend:
Company A: 48222.41
Company B: 52894.19
Company C: 60170.29

Equilibrium Market Shares:
Company A: 37.44%
Company B: 35.64%
Company C: 26.92%

Equilibrium Profits:
Company A: 316433.39
Company B: 298163.27
Company C: 209902.05